In [ ]:
#installing a necessary library
!pip3 install pyod

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from pyod.models.hbos import HBOS

In [ ]:
#loading dataset
df_compiled = pd.read_csv('/content/drive/My Drive/Com-Olho-Datasets/Unstructured_Data/Research_Extract_Columns_Compiled.csv')
df_compiled.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,time,time-domContentLoadedEventStart,time-domLoading,unload_ms,diff_time_dcle_end-start,diff_time_response_end-start,GEO_CITY_0,GEO_COUNTRY_0,GEO_IP_0,GEO_LAT_0,GEO_LNG_0,GEO_REGION_0,GEO_AS_0,connection,custom_req_date,custom_req_time,fingerprint_client,framerate,mem-jsHeapSizeLimit,mem-totalJSHeapSize,mem-usedJSHeapSize,AndroidVersion,Model,m_x,m_y,m_z,o_x,o_y,o_z,IsMotionTrue,sendBeacon,clock,globals-time,sort,video
0,6,1336.0,26,19251,20.0,19,None,Argentina,186.12.57.50,-34.6407,-58.5638,Buenos Aires,Techtel LMDS,cellular,5/29/2019,20:13,d1e03df72ce1bebd9760bce2545a62c1,20,491.0,17.1,16.1,8.1.0,SM-J701M,None,None,None,None,None,None,0.0,1.0,236.0,4.400,137.9,360x640x24
1,15,2917.0,32,3118,0.0,15,Johannesburg,South Africa,105.12.7.2,-26.2309,28.0583,Gauteng,CELL-C,cellular,5/29/2019,20:01,1635d10e5fb94420d89d91771a2309ef,20,553.0,10.0,10.0,7,LG-M250,None,None,None,123.1435301,6.141298159,-4.78748806,0.0,1.0,467.0,10.400,139.5,424x753x24
2,12,2001.0,15,26490,0.0,2,Santiago,Chile,190.107.228.128,-33.4513,-70.6653,Santiago Metropolitan,Centennial Cayman,cellular,5/29/2019,21:11,d72906679f7910afe1eb13f9a6eb0a64,30,304.0,15.2,10.6,5.1,HUAWEI,None,None,None,None,None,None,0.0,1.0,198.0,8.635,1679.53,320x570x32
3,7,1410.0,34,22658,65.0,47,Santiago,Chile,191.116.234.246,-33.4513,-70.6653,Santiago Metropolitan,CLARO CHILE,cellular,5/29/2019,21:12,b37fcb261d690cdbc0109a6322294122,20,521.0,10.0,10.0,8.1.0,Moto,1.503626585,-0.631460667,2.124106646,7.759507238,56.51144993,45.13328781,1.0,1.0,373.0,4.100,108.7,360x640x24
4,5,813.0,36,None,0.0,30,None,Honduras,45.7.236.90,16.3,-86.55,Bay Islands,CABLECOLOR S.A.,wifi,5/29/2019,20:22,b57c8a72174fa1ae66c7e28a6796c76d,30,1070.0,13.4,10.0,8.0.0,SM-G930U,-0.103394389,0.072358131,0.130429268,80.29940826,39.71229548,14.23260824,1.0,1.0,215.0,1.400,397.2,360x640x24


In [ ]:
#shuffling dataset
df_shuffled = df_compiled.sample(frac=1)
df_shuffled = df_shuffled.reset_index()
df_shuffled = df_shuffled.drop(['index'],axis=1)
df_shuffled.head()

,time,time-domContentLoadedEventStart,time-domLoading,unload_ms,diff_time_dcle_end-start,diff_time_response_end-start,GEO_CITY_0,GEO_COUNTRY_0,GEO_IP_0,GEO_LAT_0,GEO_LNG_0,GEO_REGION_0,GEO_AS_0,connection,custom_req_date,custom_req_time,fingerprint_client,framerate,mem-jsHeapSizeLimit,mem-totalJSHeapSize,mem-usedJSHeapSize,AndroidVersion,Model,m_x,m_y,m_z,o_x,o_y,o_z,IsMotionTrue,sendBeacon,clock,globals-time,sort,video
0,40,1550.0,65,None,0.0,50,Ibotirama,Brazil,179.106.169.24,-11.8862,-43.2529,Bahia,PROTEL NET,wifi,6/18/2019,14:28,d2d2428d1be1bd8632105c164a6a68fc,20,747.0,10.0,10.0,9,moto,0.001612365,0.010490417,0.024070621,55.13821498,96.16338966,34.91215969,1.0,1.0,456.0,3.7,116.3,360x640x24
1,9,1392.0,43,17752,1.0,47,Pinheiro,Brazil,177.66.195.6,-2.6991,-45.1224,Maranhao,PAULO DE,wifi,06-08-19 0:00,6:58:00,fd3ba9b41df342dc4994a84689e48789,20,1008.0,20.5,13.4,9,SM-G950F,0.014840484,0.006932974,0.147902489,1.095834045,5.858804507,-0.755922658,1.0,1.0,166.0,9.7,69.7,360x740x24
2,5,1996.0,34,3872,141.0,28,Bengaluru,India,106.206.12.210,12.9833,77.5833,Karnataka,Bharti Airtel,cellular,06-03-19 0:00,5:54:00,59a01c5b3e7c8ce5fd7631fb07b63f7d,20,793.0,19.3,10.0,8.1.0,SM-J730GM,0.047762811,0.246295929,0.111854076,6.61709948,60.81878957,-11.36943559,1.0,1.0,341.0,4.5,1059.2,412x732x24
3,6,1615.0,22,54679,0.0,27,None,Argentina,170.51.148.44,-31.4135,-64.1811,Cordoba,Techtel LMDS,cellular,6/19/2019,4:37,125fbb2d1da71b0e7b95b668a47eee4e,30,793.0,14.3,11.9,8.1.0,SM-G610M,None,None,None,None,None,None,0.0,1.0,320.0,3.4,93.8,320x569x24
4,7,2204.0,27,3027,131.0,33,Kolkata,India,2405:204:43a1:b94d::1705:f8a1,22.572,88.367,West Bengal,Reliance Jio,cellular,6/15/2019,9:23,3b3db1f249097c8b7d7e3720e995cf4c,20,553.0,10.0,10.0,7,Moto,None,None,None,None,None,None,0.0,1.0,175.0,6.9,154.5,424x753x24


In [ ]:
#replace "None" in entire dataset with NAN
df_replacenone = df_shuffled.replace("None",np.nan)
df_replacenone.head()

,time,time-domContentLoadedEventStart,time-domLoading,unload_ms,diff_time_dcle_end-start,diff_time_response_end-start,GEO_CITY_0,GEO_COUNTRY_0,GEO_IP_0,GEO_LAT_0,GEO_LNG_0,GEO_REGION_0,GEO_AS_0,connection,custom_req_date,custom_req_time,fingerprint_client,framerate,mem-jsHeapSizeLimit,mem-totalJSHeapSize,mem-usedJSHeapSize,AndroidVersion,Model,m_x,m_y,m_z,o_x,o_y,o_z,IsMotionTrue,sendBeacon,clock,globals-time,sort,video
0,40,1550.0,65,NaN,0.0,50,Ibotirama,Brazil,179.106.169.24,-11.8862,-43.2529,Bahia,PROTEL NET,wifi,6/18/2019,14:28,d2d2428d1be1bd8632105c164a6a68fc,20,747.0,10.0,10.0,9,moto,0.001612365,0.010490417,0.024070621,55.13821498,96.16338966,34.91215969,1.0,1.0,456.0,3.7,116.3,360x640x24
1,9,1392.0,43,17752,1.0,47,Pinheiro,Brazil,177.66.195.6,-2.6991,-45.1224,Maranhao,PAULO DE,wifi,06-08-19 0:00,6:58:00,fd3ba9b41df342dc4994a84689e48789,20,1008.0,20.5,13.4,9,SM-G950F,0.014840484,0.006932974,0.147902489,1.095834045,5.858804507,-0.755922658,1.0,1.0,166.0,9.7,69.7,360x740x24
2,5,1996.0,34,3872,141.0,28,Bengaluru,India,106.206.12.210,12.9833,77.5833,Karnataka,Bharti Airtel,cellular,06-03-19 0:00,5:54:00,59a01c5b3e7c8ce5fd7631fb07b63f7d,20,793.0,19.3,10.0,8.1.0,SM-J730GM,0.047762811,0.246295929,0.111854076,6.61709948,60.81878957,-11.36943559,1.0,1.0,341.0,4.5,1059.2,412x732x24
3,6,1615.0,22,54679,0.0,27,NaN,Argentina,170.51.148.44,-31.4135,-64.1811,Cordoba,Techtel LMDS,cellular,6/19/2019,4:37,125fbb2d1da71b0e7b95b668a47eee4e,30,793.0,14.3,11.9,8.1.0,SM-G610M,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,320.0,3.4,93.8,320x569x24
4,7,2204.0,27,3027,131.0,33,Kolkata,India,2405:204:43a1:b94d::1705:f8a1,22.572,88.367,West Bengal,Reliance Jio,cellular,6/15/2019,9:23,3b3db1f249097c8b7d7e3720e995cf4c,20,553.0,10.0,10.0,7,Moto,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,175.0,6.9,154.5,424x753x24


In [ ]:
#Filling NAN of features columns with Median of that column
lst = ['time','time-domLoading','time-domContentLoadedEventStart','unload_ms','diff_time_dcle_end-start','diff_time_response_end-start','m_x','m_y','m_z','clock','globals-time','sort']
for i in lst:
    df_replacenone[i].fillna((df_replacenone[i].median()), inplace=True)
df_replacenone.head()

,time,time-domContentLoadedEventStart,time-domLoading,unload_ms,diff_time_dcle_end-start,diff_time_response_end-start,GEO_CITY_0,GEO_COUNTRY_0,GEO_IP_0,GEO_LAT_0,GEO_LNG_0,GEO_REGION_0,GEO_AS_0,connection,custom_req_date,custom_req_time,fingerprint_client,framerate,mem-jsHeapSizeLimit,mem-totalJSHeapSize,mem-usedJSHeapSize,AndroidVersion,Model,m_x,m_y,m_z,o_x,o_y,o_z,IsMotionTrue,sendBeacon,clock,globals-time,sort,video
0,40,1550.0,65,19929,0.0,50,Ibotirama,Brazil,179.106.169.24,-11.8862,-43.2529,Bahia,PROTEL NET,wifi,6/18/2019,14:28,d2d2428d1be1bd8632105c164a6a68fc,20,747.0,10.0,10.0,9,moto,0.001612365,0.010490417,0.024070621,55.13821498,96.16338966,34.91215969,1.0,1.0,456.0,3.7,116.3,360x640x24
1,9,1392.0,43,17752,1.0,47,Pinheiro,Brazil,177.66.195.6,-2.6991,-45.1224,Maranhao,PAULO DE,wifi,06-08-19 0:00,6:58:00,fd3ba9b41df342dc4994a84689e48789,20,1008.0,20.5,13.4,9,SM-G950F,0.014840484,0.006932974,0.147902489,1.095834045,5.858804507,-0.755922658,1.0,1.0,166.0,9.7,69.7,360x740x24
2,5,1996.0,34,3872,141.0,28,Bengaluru,India,106.206.12.210,12.9833,77.5833,Karnataka,Bharti Airtel,cellular,06-03-19 0:00,5:54:00,59a01c5b3e7c8ce5fd7631fb07b63f7d,20,793.0,19.3,10.0,8.1.0,SM-J730GM,0.047762811,0.246295929,0.111854076,6.61709948,60.81878957,-11.36943559,1.0,1.0,341.0,4.5,1059.2,412x732x24
3,6,1615.0,22,54679,0.0,27,NaN,Argentina,170.51.148.44,-31.4135,-64.1811,Cordoba,Techtel LMDS,cellular,6/19/2019,4:37,125fbb2d1da71b0e7b95b668a47eee4e,30,793.0,14.3,11.9,8.1.0,SM-G610M,0,0.00909926,0.0173,NaN,NaN,NaN,0.0,1.0,320.0,3.4,93.8,320x569x24
4,7,2204.0,27,3027,131.0,33,Kolkata,India,2405:204:43a1:b94d::1705:f8a1,22.572,88.367,West Bengal,Reliance Jio,cellular,6/15/2019,9:23,3b3db1f249097c8b7d7e3720e995cf4c,20,553.0,10.0,10.0,7,Moto,0,0.00909926,0.0173,NaN,NaN,NaN,0.0,1.0,175.0,6.9,154.5,424x753x24


In [ ]:
#seperating the features from the entire data
df_features = df_replacenone[df_replacenone.columns.intersection(lst)]
df_features.head()

,time,time-domContentLoadedEventStart,time-domLoading,unload_ms,diff_time_dcle_end-start,diff_time_response_end-start,m_x,m_y,m_z,clock,globals-time,sort
0,40,1550.0,65,19929,0.0,50,0.001612365,0.010490417,0.024070621,456.0,3.7,116.3
1,9,1392.0,43,17752,1.0,47,0.014840484,0.006932974,0.147902489,166.0,9.7,69.7
2,5,1996.0,34,3872,141.0,28,0.047762811,0.246295929,0.111854076,341.0,4.5,1059.2
3,6,1615.0,22,54679,0.0,27,0,0.00909926,0.0173,320.0,3.4,93.8
4,7,2204.0,27,3027,131.0,33,0,0.00909926,0.0173,175.0,6.9,154.5


In [ ]:
#standardizing the features
scaler=StandardScaler()
scaler.fit(df_features)
df_features_standardized = scaler.transform(df_features)
print(df_features_standardized)

[[ 0.8659061  -0.12203994  0.46691294 ... -0.24761905 -0.15533061
  -0.14465333]
 [ 0.0310596  -0.25695006  0.1419679  ... -1.12456654  0.89795658
  -0.31189357]
 [-0.07666253  0.25878231  0.00903584 ... -0.59537409 -0.01489232
   3.2392699 ]
 ...
 [ 0.8659061   2.75461957  0.4964534  ... -0.58327826  0.04654944
   3.91278869]
 [-0.07666253 -0.467       0.34875111 ...  1.05570636 -0.59420027
  -0.42673665]
 [-0.07666253 -1.06555693  0.21581905 ...  0.01848916 -0.24310454
  -0.30758696]]


In [ ]:
#Final Append to full data
def append_col(preds,df_replacenone,algotype):
    finallst = []
    if algotype == "lof":
        for i in preds:
            if i == 1:
                val = "n"
            else:
                val = "o"
            finallst.append(val)
    elif algotype == "hbos":
        for i in preds:
            if i == 0:
                val = "n"
            else:
                val = "o"
            finallst.append(val)
    colname = "TV_"+algotype
    df_replacenone[colname] = finallst
    print("Final TV column appended for "+algotype)
    return df_replacenone

In [ ]:
#Output Data
def final_output(preds,algotype):
    print("Below is the count of inliers and outliers for "+algotype+" on the unstructured dataset")
    outt = []
    for k in preds:
        outt.append(k)
    if algotype == "lof":
        print("Number of inliers = "+str(outt.count(1)))
        print("Number of outliers = "+str(outt.count(-1)))
    elif algotype == "hbos":
        print("Number of inliers = "+str(outt.count(0)))
        print("Number of outliers = "+str(outt.count(1)))

In [ ]:
#LOF Algorithm
def LOF_algorithm(df_features_standardized,df_replacenone):
    print("----------Starting LOF ALGORTIHM----------")
    clf = LocalOutlierFactor(n_neighbors=20)
    preds = clf.fit_predict(df_features_standardized)
    final_output(preds,'lof')
    df_replacenone = append_col(preds,df_replacenone,"lof")
    return df_replacenone

In [ ]:
#HBOS Algorithm
def HBOS_algorithm(df_features_standardized,df_replacenone):
    print("----------Starting HBOS ALGORTIHM----------")
    clf = HBOS(n_bins=20)
    clf.fit(df_features_standardized)    
    preds = clf.predict(df_features_standardized)
    final_output(preds,'hbos')
    df_replacenone = append_col(preds,df_replacenone,"hbos")
    return df_replacenone

In [ ]:
#my main function
df_replacenone = LOF_algorithm(df_features_standardized,df_replacenone) 
print("\n")
df_replacenone = HBOS_algorithm(df_features_standardized,df_replacenone)
print("\n")
df_replacenone.head()

----------Starting LOF ALGORTIHM----------
Below is the count of inliers and outliers for lof on the unstructured dataset
Number of inliers = 38309
Number of outliers = 2846
Final TV column appended for lof


----------Starting HBOS ALGORTIHM----------
Below is the count of inliers and outliers for hbos on the unstructured dataset
Number of inliers = 37050
Number of outliers = 4105
Final TV column appended for hbos




,time,time-domContentLoadedEventStart,time-domLoading,unload_ms,diff_time_dcle_end-start,diff_time_response_end-start,GEO_CITY_0,GEO_COUNTRY_0,GEO_IP_0,GEO_LAT_0,GEO_LNG_0,GEO_REGION_0,GEO_AS_0,connection,custom_req_date,custom_req_time,fingerprint_client,framerate,mem-jsHeapSizeLimit,mem-totalJSHeapSize,mem-usedJSHeapSize,AndroidVersion,Model,m_x,m_y,m_z,o_x,o_y,o_z,IsMotionTrue,sendBeacon,clock,globals-time,sort,video,TV_lof,TV_hbos
0,40,1550.0,65,19929,0.0,50,Ibotirama,Brazil,179.106.169.24,-11.8862,-43.2529,Bahia,PROTEL NET,wifi,6/18/2019,14:28,d2d2428d1be1bd8632105c164a6a68fc,20,747.0,10.0,10.0,9,moto,0.001612365,0.010490417,0.024070621,55.13821498,96.16338966,34.91215969,1.0,1.0,456.0,3.7,116.3,360x640x24,n,n
1,9,1392.0,43,17752,1.0,47,Pinheiro,Brazil,177.66.195.6,-2.6991,-45.1224,Maranhao,PAULO DE,wifi,06-08-19 0:00,6:58:00,fd3ba9b41df342dc4994a84689e48789,20,1008.0,20.5,13.4,9,SM-G950F,0.014840484,0.006932974,0.147902489,1.095834045,5.858804507,-0.755922658,1.0,1.0,166.0,9.7,69.7,360x740x24,n,n
2,5,1996.0,34,3872,141.0,28,Bengaluru,India,106.206.12.210,12.9833,77.5833,Karnataka,Bharti Airtel,cellular,06-03-19 0:00,5:54:00,59a01c5b3e7c8ce5fd7631fb07b63f7d,20,793.0,19.3,10.0,8.1.0,SM-J730GM,0.047762811,0.246295929,0.111854076,6.61709948,60.81878957,-11.36943559,1.0,1.0,341.0,4.5,1059.2,412x732x24,n,o
3,6,1615.0,22,54679,0.0,27,NaN,Argentina,170.51.148.44,-31.4135,-64.1811,Cordoba,Techtel LMDS,cellular,6/19/2019,4:37,125fbb2d1da71b0e7b95b668a47eee4e,30,793.0,14.3,11.9,8.1.0,SM-G610M,0,0.00909926,0.0173,NaN,NaN,NaN,0.0,1.0,320.0,3.4,93.8,320x569x24,n,n
4,7,2204.0,27,3027,131.0,33,Kolkata,India,2405:204:43a1:b94d::1705:f8a1,22.572,88.367,West Bengal,Reliance Jio,cellular,6/15/2019,9:23,3b3db1f249097c8b7d7e3720e995cf4c,20,553.0,10.0,10.0,7,Moto,0,0.00909926,0.0173,NaN,NaN,NaN,0.0,1.0,175.0,6.9,154.5,424x753x24,n,o


In [ ]:
df_replacenone.to_csv("finallabeleddata_new.csv")